# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: VIVEK AGGARWAL
#### Student ID: S4015465

Date: 04/09/2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

## Introduction

The offered code is a Python script that seems to preprocess and clean data for a dataset that is stored in a CSV file with the name "s4015465_dataset2.csv." This dataset is combined by the code with another dataset that is located in "s4015465_dataset1_solution.csv."

##  Import libraries 

In [1]:
## Importing libraries
import pandas as pd
import numpy as np
import xml.etree.ElementTree as etree 
import re
import random
import string


### 1. Examining and loading data


In [2]:
df = pd.read_csv('s4015465_dataset2.csv')

In [3]:
df.head()

Monthly Payment              Closing               Organisation  \
0          1200.00  2012-12-27 12:00:00         Protocol Education   
1          3125.00  2013-08-25 15:00:00                Cordius Ltd   
2          4333.33  2013-01-16 15:00:00  Understanding Recruitment   
3          3750.00  2013-02-13 15:00:00     Wallace Hind Selection   
4          2916.67  2013-01-12 12:00:00        Greenhill Group Ltd   

           Location                Category  Full-Time Equivalent (FTE)  \
0                UK               Education                         0.8   
1                UK  Information Technology                         1.0   
2  Walton-On-Thames  Information Technology                         1.0   
3                UK                   Sales                         1.0   
4   Eastern England                   Sales                         1.0   

               Opening                                          Job Title  \
0  2012-11-27 12:00:00             Higher Level Teaching Assistant (HLTA)   
1  2013-06-26 15:00:00  Software Developers / Software Architects  Not...   
2  2012-10-18 15:00:00                      Senior BI Consultant (Cognos)   
3  2012-12-15 15:00:00  Technical Sales Manager, Pigments Fillers to S...   
4  2012-12-29 12:00:00                  Channel Account Manager  Software   

        Type  
0        NaN  
1  Permanent  
2  Permanent  
3        NaN  
4        NaN

In [4]:
df.shape

(5000, 9)

### 2. Resolving schema conflicts


In [5]:
df.rename(columns={'Monthly Payment':'Salary'},inplace=True) 
df.rename(columns={'Closing':'CloseDate'},inplace=True)
df.rename(columns={'Organisation':'Company'},inplace=True)
df.rename(columns={'Full-Time Equivalent (FTE)':'ContractTime'},inplace=True)
df.rename(columns={'Opening':'OpenDate'},inplace=True)
df.rename(columns={'Type':'ContractType'},inplace=True)
df.rename(columns={'Job Title':'Title'},inplace=True)

In [6]:
df.head()

Salary            CloseDate                    Company          Location  \
0  1200.00  2012-12-27 12:00:00         Protocol Education                UK   
1  3125.00  2013-08-25 15:00:00                Cordius Ltd                UK   
2  4333.33  2013-01-16 15:00:00  Understanding Recruitment  Walton-On-Thames   
3  3750.00  2013-02-13 15:00:00     Wallace Hind Selection                UK   
4  2916.67  2013-01-12 12:00:00        Greenhill Group Ltd   Eastern England   

                 Category  ContractTime             OpenDate  \
0               Education           0.8  2012-11-27 12:00:00   
1  Information Technology           1.0  2013-06-26 15:00:00   
2  Information Technology           1.0  2012-10-18 15:00:00   
3                   Sales           1.0  2012-12-15 15:00:00   
4                   Sales           1.0  2012-12-29 12:00:00   

                                               Title ContractType  
0             Higher Level Teaching Assistant (HLTA)          NaN  
1  Software Developers / Software Architects  Not...    Permanent  
2                      Senior BI Consultant (Cognos)    Permanent  
3  Technical Sales Manager, Pigments Fillers to S...          NaN  
4                  Channel Account Manager  Software          NaN

#### Conflict 1: XXXX (e.g. naming conflicts in xx column and xx column)

Differences in column names can cause serious schema conflicts when they exist between datasets. As an illustration, one dataset might use "Salary" as the column name for pay, whereas another might use "Monthly Payment." Column names must be standardized.

Missing Columns: Different datasets may have varying numbers of columns, some of which may be present in one dataset but absent in another. When merging, choices must be made regarding whether to include missing columns or leave them out.

Columns having the same name in various datasets may have columns with distinct data types. For instance, a "Date" column might be stored in one dataset as a string and in another as a datetime object. Consistent data types must be maintained.

In [7]:
# Define the to_clean function
def to_clean(text):
    original_text = text
    c_text = re.sub(r'\s+', ' ', text.strip())
    c_text = re.sub(r'[^\w\s.]+', '', c_text)
    c_text = c_text.replace('*', '')
    c_text = re.sub(r'\b[kK]\b', '', c_text)
    c_text = re.sub(r'\s*â€“\s*', ' ', c_text)
    c_text = re.sub(r'[^\w\s,\/\+\+]+', '', c_text)



    modifications = []

    if c_text != original_text:
        if len(c_text) < len(original_text):
            modifications.append(f'Removed characters: {original_text[len(c_text):]}')
        elif len(c_text) > len(original_text):
            modifications.append(f'Added characters: {c_text[len(original_text):]}')
        else:
            for o, c in zip(original_text, c_text):
                if o != c:
                    modifications.append(f'Replaced "{o}" with "{c}"')

        return original_text, c_text, ' '.join(modifications)
    else:
        return None, c_text, None


# Replace 'NA' and empty spaces with 'non specified' in ContractType and ContractTime columns
df['ContractType'].replace(['NA', ''], 'non specified', inplace=True)
#df['ContractTime'].replace(['NA', ''], 'non specified', inplace=True)


# Replace '-', 'N/A', and blanks with 'non specified' in ContractType column
df['ContractType'].replace(['-', 'N/A', '', ' '], 'non specified', inplace=True)
df['ContractTime'].replace(['-', 'N/A', '', ' '], 'non specified', inplace=True)

# Save the cleaned DataFrame to a CSV file
df.to_csv('cleaned_data.csv', index=False)

print(f'Cleaned data saved to cleaned_data.csv')
print(df.head())
print(df.info())


Cleaned data saved to cleaned_data.csv
    Salary            CloseDate                    Company          Location  \
0  1200.00  2012-12-27 12:00:00         Protocol Education                UK   
1  3125.00  2013-08-25 15:00:00                Cordius Ltd                UK   
2  4333.33  2013-01-16 15:00:00  Understanding Recruitment  Walton-On-Thames   
3  3750.00  2013-02-13 15:00:00     Wallace Hind Selection                UK   
4  2916.67  2013-01-12 12:00:00        Greenhill Group Ltd   Eastern England   

                 Category  ContractTime             OpenDate  \
0               Education           0.8  2012-11-27 12:00:00   
1  Information Technology           1.0  2013-06-26 15:00:00   
2  Information Technology           1.0  2012-10-18 15:00:00   
3                   Sales           1.0  2012-12-15 15:00:00   
4                   Sales           1.0  2012-12-29 12:00:00   

                                               Title ContractType  
0             Higher Level 

Different data formats can be found within columns, including date formats, monetary symbols, and numeric representations. For accurate analysis, it is necessary to standardize these variations.

Categorical Variables: There may be discrepancies in the categories utilized or the encoding of categorical values if the datasets contain categorical variables (such as employment classifications or contract types). It is vital to standardize these categories.

Different primary keys or unique identifiers may be used for various datasets. During merging, it is crucial to resolve conflicts involving unique identifiers to maintain the integrity of the data.

Data Units: Quantitative values shown in columns may be expressed in a variety of measuring units, such as dollars or pounds. It's crucial to convert data into a consistent unit.


In [8]:
df.shape

(5000, 9)

In [9]:

# Create a new column 'SrcName' filled with 'Non-specified'
df['SrcName'] = 'Non-specified'

# Create a new column 'PositionID' with 8 random numbers
def generate_random_position_id():
    return ''.join(random.choices(string.digits, k=8))

df['PositionID'] = df.apply(lambda row: generate_random_position_id(), axis=1)

# Print the updated DataFrame
print(df)


       Salary            CloseDate                       Company  \
0     1200.00  2012-12-27 12:00:00            Protocol Education   
1     3125.00  2013-08-25 15:00:00                   Cordius Ltd   
2     4333.33  2013-01-16 15:00:00     Understanding Recruitment   
3     3750.00  2013-02-13 15:00:00        Wallace Hind Selection   
4     2916.67  2013-01-12 12:00:00           Greenhill Group Ltd   
...       ...                  ...                           ...   
4995  5833.33  2013-03-31 00:00:00  Jonathan Lee Recruitment Ltd   
4996  5000.00  2012-08-27 12:00:00                    Impact 360   
4997  2833.33  2013-07-23 00:00:00       Aspire Data Recruitment   
4998  4791.67  2012-12-18 15:00:00              Annapurna HR Ltd   
4999  3541.67  2013-10-20 12:00:00   Parallel Consulting Limited   

              Location                Category  ContractTime  \
0                   UK               Education           0.8   
1                   UK  Information Technology         

#### Conflict 2: XXXX
......

### 3. Merging data



In [10]:
df_old = pd.read_csv('s4015465_dataset1_solution.csv')


In [11]:
df_old.head()

PositionID                                              Title    Location  \
0    69568721          Manufacturing Engineer Injection Moulding   Hampshire   
1    71351976                    Civil Design Engineer London UK      London   
2    68303946  Project ManagerTeam LeadTechnology ChangeFinan...      London   
3    72654312                      2nd  3rd Line Support Windows  Birmingham   
4    71799126                      Marketing Executive ecommerce   Berkshire   

                     Company  Salary ContractType ContractTime  \
0          Austin Fraser Ltd   37500    full_time    permanent   
1             Selby Jennings   32500    full_time    permanent   
2         Cornwallis Elt Ltd   67500    full_time    permanent   
3                Harvey Nash   30000    full_time    permanent   
4  ECS Recruitment Group Ltd   32500    full_time    permanent   

                    Category             OpenDate            CloseDate  \
0           Engineering Jobs  2013-06-26 00:00:00  2013-09-24 00:00:00   
1           Engineering Jobs  2012-12-24 15:00:00  2013-01-23 15:00:00   
2  Accounting & Finance Jobs  2012-03-10 00:00:00  2012-06-08 00:00:00   
3                    IT Jobs  2013-12-07 00:00:00  2014-03-07 00:00:00   
4                 Sales Jobs  2013-12-07 12:00:00  2014-02-05 12:00:00   

                    SrcName  
0  energygenerationjobs.com  
1  energygenerationjobs.com  
2  energygenerationjobs.com  
3  energygenerationjobs.com  
4  energygenerationjobs.com

In [12]:
# Code to merge two data sets
df1 = pd.read_csv('cleaned_data.csv')
dff_merged_newclean = pd.concat([df_old, df])

In [13]:
dff_merged_newclean.shape

(55753, 11)

### 4. Resolving data conflicts:


#### Conflicts


The datasets being combined may include the same information in columns with different names. This problem is fixed by renaming columns, as done in the code.

Missing Values: One or both datasets may contain some columns with missing values. It is necessary to deal with these missing values, either by adding the proper values or by labeling them as "non specified."

Data Type Inconsistencies: In the two datasets, columns with the same name may have columns with various data types. In one dataset, a number column might be saved as text, whereas in another, it might be kept as integers. To maintain uniform data types, it is necessary to address data type inconsistencies.



Inconsistent Categorization: Categorical variables, such as contract types or job categories, may have different naming conventions or categories in the two datasets. These need to be standardized for consistency.

Duplicate Data: Duplicate records may exist in one or both datasets. Identifying and removing duplicates is essential to maintain data integrity.

Data Formatting Issues: Inconsistent formatting of text data, such as extra spaces, special characters, or variations in capitalization, can create conflicts. Standardizing text data is necessary to avoid these issues.

In [14]:
dff_merged_newclean.isnull().sum()

PositionID         0
Title              1
Location           0
Company         5422
Salary             0
ContractType    1420
ContractTime       0
Category           0
OpenDate           0
CloseDate          0
SrcName            0
dtype: int64

In [15]:
# Fill missing values in the 'Company' and 'ContractType' columns with 'Non-specified'
columns_to_fill = ['Company', 'ContractType','Title']
dff_merged_newclean[columns_to_fill] = dff_merged_newclean[columns_to_fill].fillna('Non-specified')


In [16]:
# Check for missing values in the DataFrame 'df'
missing_values = dff_merged_newclean.isnull().sum()
print(missing_values)


PositionID      0
Title           0
Location        0
Company         0
Salary          0
ContractType    0
ContractTime    0
Category        0
OpenDate        0
CloseDate       0
SrcName         0
dtype: int64


......

#### Finding global key for the data

Identifying a global key, as done in the code, can be challenging. A global key is a unique identifier that ensures data integrity during merging. The absence of a suitable global key may lead to difficulties in merging and deduplication.

In [17]:


# Define a list of candidate columns to check for uniqueness
candidate_columns = ['PositionID', 'Title', 'Location', 'Company', 'Salary', 'ContractType', 'ContractTime', 'Category', 'OpenDate', 'CloseDate', 'SrcName']

# Iterate through candidate columns and check uniqueness
global_key = None
for column in candidate_columns:
    if dff_merged_newclean[column].is_unique:
        global_key = column
        break

# Check if a global key was found
if global_key:
    print(f"The global key is '{global_key}'.")
else:
    print("No suitable global key found.")


The global key is 'PositionID'.


.......

### 5. Saving the integrated and reshaped data


In [18]:
# code to save output data
# Save the DataFrame with missing values filled to a CSV file
dff_merged_newclean.to_csv('s4015465dataset_integrated.csv', index=False)


## Summary of the Assessment Task

In conclusion, this program imports a CSV dataset, cleans and standardizes the data, creates new columns, merges it with an existing dataset, handles missing values, and looks for a global key. A new CSV file is created and saved with the finished, combined, and cleaned dataset for use or analysis. This code aids in ensuring that the data is presented for downstream processes in a more organized and consistent manner.
